# Draw a world map with population data
- source: population data in JSON file(s)
- target: a graph show population of countries around the world

## Process JSON data

In [33]:
# import JSON file
import json


# Load the data into a list.
filename = 'data\population_data.json'
with open(filename) as f:
    pop_data = json.load(f)


In [34]:
from pygal_maps_world.i18n import COUNTRIES


def get_country_code(country_name):
    '''Return the Pygal 2-digit country code for the given country.'''
    for code, name in COUNTRIES.items():
        if name == country_name:
            return code
    # If the country wasn't found, return None.
    return None


In [35]:
# Build a dictionary of population data
cc_populations = {}
for pop_dict in pop_data:
    if pop_dict['Year'] == '2010':
        country_name = pop_dict['Country Name']
        population = int(float(pop_dict['Value']))
        # Prepare for pygal-maps:  
        #     country_name=>country code
        code = get_country_code(country_name)
        if code:
            cc_populations[code] = population

In [36]:
# Group the countries into 3 population levels
cc_pops_1, cc_pops_2, cc_pops_3 = {}, {}, {}
for cc, pop in cc_populations.items():
    if pop < 10000000:
        cc_pops_1[cc] = pop
    elif pop < 1000000000:
        cc_pops_2[cc] = pop
    else:
        cc_pops_3[cc] = pop

## Draw the World Map

In [37]:
from pygal.style import LightColorizedStyle as LCS, \
                        RotateStyle as RS 
import pygal_maps_world.maps


# Style select: base-color & base-style
#  - base style: use lighter color to fill the pygal-graph
wm_style = RS('#ccd865', base_style=LCS)
# Use style to draw map[wm]
wm = pygal_maps_world.maps.World(style=wm_style)

# Map format
wm.title = 'World Population in 2010, by Country'
wm.add('0-10m', cc_pops_1)   # add series
wm.add('10m-1bn', cc_pops_2)
wm.add('>1bn', cc_pops_3)

# Render to SVG file
wm.render_to_file('output\world_population.svg')